In [4]:
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
from autogluon.tabular import TabularDataset, TabularPredictor
import xgboost as xgb
from catboost import CatBoostRegressor
from sklearn.model_selection import GridSearchCV


train_df = pd.read_csv('./train.csv')
test_df = pd.read_csv('./test.csv')
sample_submission = pd.read_csv('./sample_submission.csv')
building_df = pd.read_csv('./building_info.csv')

from datetime import datetime, timedelta

def classify_weekdays(start_date, end_date):
    date_format = "%Y-%m-%d"
    start = datetime.strptime(start_date, date_format)
    end = datetime.strptime(end_date, date_format)
    
    classification = []
    current_date = start
    
    while current_date <= end:
        if current_date.weekday() < 5:  # 0: 월요일, 1: 화요일, ..., 4: 금요일
            classification.append(0)  # 평일
        else:
            classification.append(1)  # 주말
        
        current_date += timedelta(days=1)
    
    return classification

# 예시 데이터 프레임 생성
data = {'date': pd.date_range(start='2022-06-01', end='2022-08-24')}
train = pd.DataFrame(data)
t_data = {'date': pd.date_range(start='2022-08-25', end='2022-08-31')}
test = pd.DataFrame(t_data)
# 여름 계절성 정보 추가
train['summer'] = train['date'].apply(lambda x: 1 if (x.month >= 6) and (x.month <= 8) else 0)
test['summer'] = test['date'].apply(lambda x: 1 if (x.month >= 6) and (x.month <= 8) else 0)

# 주말/평일 분류한 결과를 'holidays' 열에 추가
start_date = "2022-06-01"
end_date = "2022-08-24"
test_start_date = "2022-08-25"
test_end_date = "2022-08-31"


train['holidays'] = classify_weekdays(start_date, end_date)
# '2022-08-15'는 주말이 아닌 휴일로 설정
train.loc[train['date'] == '2022-08-15', 'holidays'] = 2

train_df['holidays'] = train['holidays']
test['holidays'] = classify_weekdays(test_start_date, test_end_date)
test_df['holidays'] = test['holidays']
#added
train_df['일시'] = pd.to_datetime(train_df['일시'], format='%Y%m%d %H')
test_df['일시'] = pd.to_datetime(test_df['일시'], format='%Y%m%d %H')

# date time feature 생성
train_df['hour'] = train_df['일시'].dt.hour
train_df['day'] = train_df['일시'].dt.day
train_df['month'] = train_df['일시'].dt.month
train_df['year'] = train_df['일시'].dt.year

test_df['hour'] = test_df['일시'].dt.hour
test_df['day'] = test_df['일시'].dt.day
test_df['month'] = test_df['일시'].dt.month
test_df['year'] = test_df['일시'].dt.year

train_df['sin_time'] = np.sin(2*np.pi*train_df.hour/24)
train_df['cos_time'] = np.cos(2*np.pi*train_df.hour/24)

test_df['sin_time'] = np.sin(2*np.pi*test_df.hour/24)
test_df['cos_time'] = np.cos(2*np.pi*test_df.hour/24)
train_df['THI'] = 9/5*train_df['기온(C)'] - 0.55*(1-train_df['습도(%)']/100)*(9/5*train_df['습도(%)']-26)+32
test_df['THI'] = 9/5*test_df['기온(C)'] - 0.55*(1-test_df['습도(%)']/100)*(9/5*test_df['습도(%)']-26)+32

# 데이터셋 생성
train_df = train_df.drop(columns=['일조(hr)', '일사(MJ/m2)', '강수량(mm)', '풍속(m/s)'])
test_df = test_df.drop(columns=['강수량(mm)', '풍속(m/s)'])
train_data = TabularDataset(train_df)
test_data = TabularDataset(test_df)

label = '전력소비량(kWh)'

# AutoGluon 모델 학습
predictor = TabularPredictor(label=label, problem_type='regression').fit(
    train_data, presets=['best_quality'], auto_stack=True, num_bag_folds=10, num_bag_sets=20, num_stack_levels=3, refit_full=True)

# XGBoost와 CatBoost 모델 초기화
xgb_model = xgb.XGBRegressor()

catboost_model = CatBoostRegressor()








No path specified. Models will be saved in: "AutogluonModels\ag-20230818_230425\"
Presets specified: ['best_quality']
Stack configuration (auto_stack=True): num_stack_levels=3, num_bag_folds=10, num_bag_sets=20
	Consider setting `time_limit` to ensure training finishes within an expected duration or experiment with a small portion of `train_data` to identify an ideal `presets` and `hyperparameters` configuration.
Beginning AutoGluon training ...
AutoGluon will save models to "AutogluonModels\ag-20230818_230425\"
AutoGluon Version:  0.8.2
Python Version:     3.9.13
Operating System:   Windows
Platform Machine:   AMD64
Platform Version:   10.0.22000
Disk Space Avail:   71.38 GB / 511.33 GB (14.0%)
Train Data Rows:    204000
Train Data Columns: 13
Label Column: 전력소비량(kWh)
Preprocessing data ...
Using Feature Generators to preprocess the data ...
Fitting AutoMLPipelineFeatureGenerator...
	Available Memory:                    4975.74 MB
	Train Data (Original)  Memory Usage: 33.98 MB (0.7% o

ValueError: 
All the 324 fits failed.
It is very likely that your model is misconfigured.
You can try to debug the error by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
324 fits failed with the following error:
Traceback (most recent call last):
  File "C:\Users\wkdal\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\sklearn\model_selection\_validation.py", line 686, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "C:\Users\wkdal\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\xgboost\core.py", line 620, in inner_f
    return func(**kwargs)
  File "C:\Users\wkdal\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\xgboost\sklearn.py", line 

In [ ]:
# XGBoost GridSearch
xgb_param_grid = {
    'n_estimators': [100, 300, 500],
    'learning_rate': [0.01, 0.1, 0.2],
    'max_depth': [3, 5, 7],
    'subsample': [0.8, 1.0],
    'colsample_bytree': [0.8, 1.0]
}

xgb_grid_search = GridSearchCV(estimator=xgb_model, param_grid=xgb_param_grid, scoring='neg_mean_squared_error', cv=3)
xgb_grid_search.fit(train_data.drop(columns=[label]), train_data[label])
best_xgb_model = xgb_grid_search.best_estimator_catboost_model = CatBoostRegressor()

In [ ]:
# 다른 모델 추가 예측
xgb_model.fit(train_data.drop(columns=[label]), train_data[label])

In [ ]:
# CatBoost GridSearch

catboost_param_grid = {
    'iterations': [100, 300, 500],
    'learning_rate': [0.01, 0.1, 0.2],
    'depth': [3, 5, 7],
    'subsample': [0.8, 1.0],
    'colsample_bylevel': [0.8, 1.0]
}

catboost_grid_search = GridSearchCV(estimator=catboost_model, param_grid=catboost_param_grid, scoring='neg_mean_squared_error', cv=3)


In [ ]:
catboost_grid_search.fit(train_data.drop(columns=[label]), train_data[label])


In [ ]:
best_catboost_model = catboost_grid_search.best_estimator_
xgb_predictions = xgb_model.predict(test_data.drop(columns=[label]))
catboost_predictions = catboost_model.predict(test_data.drop(columns=[label]))

In [ ]:
sample_submission = pd.DataFrame({'date': test_df['일시'], 'AutoGluon': predictor.predict(test_data), 'XGBoost': xgb_predictions, 'CatBoost': catboost_predictions})

sample_submission.to_csv('./ensemble_submission.csv', index=False)